In [1]:
import sys
import os

if 'google.colab' in sys.modules:
    # Клонуємо репозиторій
    if not os.path.exists('/content/NLP'):
        !git clone -b lab-03 https://github.com/AndrianaNahirna/NLP.git

    %cd /content/NLP

    # Встановлюємо бібліотеки
    !pip install -r requirements.txt -q

    #!pip install stanza tqdm -q

    sys.path.append('/content/NLP')

print("Середовище налаштовано.")

Cloning into 'NLP'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 137 (delta 46), reused 130 (delta 39), pack-reused 0 (from 0)
Receiving objects: 100% (137/137), 1.87 MiB | 9.16 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/NLP
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 50.7 MB/s eta 0:00:00
Середовище налаштовано.


In [2]:
import pandas as pd

if 'google.colab' in sys.modules:
    FOLDER_ID = '1Z4ko8PYcLJOnnU98T6MTXLVYHnpMkHVK'

    # Завантаження датасету
    os.makedirs('/content/NLP/data', exist_ok=True)
    !gdown --folder https://drive.google.com/drive/folders/{FOLDER_ID} -O /content/NLP/data/

    import glob
    csv_files = glob.glob('/content/NLP/data/**/processed_v2.csv', recursive=True)

    if csv_files:
        data_path = csv_files[0]
        df = pd.read_csv(data_path)
        print(f"Датасет завантажено. Кількість рядків: {len(df)}")
    else:
        print("Файл processed_v2.csv не знайдено.")
else:
    # Локальний шлях
    df = pd.read_csv('../data/processed_v2.csv')

Retrieving folder contents
Processing file 17odn4ukdHLvZKqqUuaTNPuHX66Aal-zk processed_v2.csv
Processing file 1tVj7OaRkYqaoVtmDGgDxUQ8nkDUvy7W7 raw.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=17odn4ukdHLvZKqqUuaTNPuHX66Aal-zk
To: /content/NLP/data/NLP_datasets/processed_v2.csv
100% 5.67M/5.67M [00:00<00:00, 28.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tVj7OaRkYqaoVtmDGgDxUQ8nkDUvy7W7
To: /content/NLP/data/NLP_datasets/raw.csv
100% 5.96M/5.96M [00:00<00:00, 135MB/s]
Download completed
Датасет завантажено. Кількість рядків: 3034


In [4]:
from sentiment.src.ling_features import LinguisticExtractor
from tqdm.auto import tqdm
import pandas as pd
import torch

original_load = torch.load
def patched_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return original_load(*args, **kwargs)
torch.load = patched_load

# Прогрес-бар
tqdm.pandas(desc="Обробка текстів (Stanza)")

# Завантажуємо модель Stanza
print("Ініціалізація Stanza NLP...")
extractor = LinguisticExtractor(use_gpu=True)
print("Stanza ініціалізована.")

# Очищаємо пусті рядки
df = df.dropna(subset=['processed_text']).reset_index(drop=True)

# Лематизація
def apply_stanza(text):
    result = extractor.extract_features(text)
    return pd.Series([result['lemma_text'], result['pos_seq'], result['pos_text']])

print("Початок лематизації та POS-тегування...")
df[['lemma_text', 'pos_seq', 'pos_text']] = df['processed_text'].progress_apply(apply_stanza)

# Зберігаємо результат
output_path = '/content/NLP/data/processed_v3_lemma.csv'
df.to_csv(output_path, index=False)

print(f"\nДані збережено у {output_path}")

# "До і Після"
print("\nПриклад результату:")
display(df[['processed_text', 'lemma_text', 'pos_seq']].head(3))

# Завантажуємо файл на комп'ютер
from google.colab import files
files.download(output_path)

Ініціалізація Stanza NLP...


Stanza ініціалізована.
Початок лематизації та POS-тегування...


Обробка текстів (Stanza):   0%|          | 0/3034 [00:00<?, ?it/s]


Дані збережено у /content/NLP/data/processed_v3_lemma.csv

Приклад результату:


,processed_text,lemma_text,pos_seq
0,В кінцевому результаті роботою данного інтерне...,в кінцевий результат робота данний інтернет - ...,ADP ADJ NOUN NOUN ADJ NOUN PUNCT NOUN VERB ADJ...
1,"12.04.16 замовив МD818 (доставили швидко, тут ...",12 . 04 . 16 замовити мd818 ( доставити швидко...,NUM PUNCT NUM PUNCT NUM VERB PROPN PUNCT VERB ...
2,"Зірвали передноворічний замовлення, зіпсували ...","зірвати передноворічний замовлення , зіпсувати...",VERB ADJ NOUN PUNCT VERB NOUN PUNCT NOUN NOUN ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

print("Побудова Baseline-моделей (Logistic Regression)")

TARGET_COL = 'target'

# Розбиваємо на train/test (80/20)
# Очищений текст
X_raw_train, X_raw_test, y_train, y_test = train_test_split(df['processed_text'], df[TARGET_COL], test_size=0.2, random_state=42)
# Леми
X_lem_train, X_lem_test, _, _ = train_test_split(df['lemma_text'], df[TARGET_COL], test_size=0.2, random_state=42)
# Очищений текст + POS-теги
df['text_with_pos'] = df['processed_text'].fillna("") + " " + df['pos_seq'].fillna("")
X_pos_train, X_pos_test, _, _ = train_test_split(df['text_with_pos'], df[TARGET_COL], test_size=0.2, random_state=42)

def train_and_eval(X_train, X_test, y_train, y_test, name, ngram_range=(1, 1)):
    vec = TfidfVectorizer(max_features=1200, ngram_range=ngram_range)
    X_train_vec = vec.fit_transform(X_train.fillna(""))
    X_test_vec = vec.transform(X_test.fillna(""))

    clf = LogisticRegression(max_iter=1000, random_state=42)
    clf.fit(X_train_vec, y_train)
    preds = clf.predict(X_test_vec)

    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds, average='macro')
    print(f"   {name}:")
    print(f"   Accuracy: {acc:.4f} | Macro-F1: {f1:.4f}\n")
    return acc, f1

# Тренуємо три моделі
acc_raw, f1_raw = train_and_eval(X_raw_train, X_raw_test, y_train, y_test, "Baseline 1 (Без лем, processed_v2)")
acc_lem, f1_lem = train_and_eval(X_lem_train, X_lem_test, y_train, y_test, "Baseline 2 (З лемами, lemma_text)")
acc_pos, f1_pos = train_and_eval(X_pos_train, X_pos_test, y_train, y_test, "Baseline 3 (processed_v2 + POS n-grams)", ngram_range=(1, 2))

print(f"Різниця Accuracy (Леми vs База): {(acc_lem - acc_raw) * 100:+.2f}%")
print(f"Різниця Accuracy (POS vs База): {(acc_pos - acc_raw) * 100:+.2f}%")

Побудова Baseline-моделей (Logistic Regression)
🔹 Baseline 1 (Без лем, processed_v2):
   Accuracy: 0.8484 | Macro-F1: 0.8253

🔹 Baseline 2 (З лемами, lemma_text):
   Accuracy: 0.8748 | Macro-F1: 0.8572

🔹 Baseline 3 (processed_v2 + POS n-grams):
   Accuracy: 0.8204 | Macro-F1: 0.7865

💡 Різниця Accuracy (Леми vs База): +2.64%
💡 Різниця Accuracy (POS vs База): -2.80%


In [30]:
print("Приклади помилок лематизації")

error_indices = [1, 2]
sample_errors = df[df['pos_seq'].str.contains('X X X', na=False) | df['processed_text'].str.contains('дрставка|празднік', na=False)].head(10)

if len(sample_errors) < 10:
    sample_errors = df.sample(10, random_state=42)

for i, row in sample_errors.iterrows():
    print(f"--- Приклад #{i} ---")
    print(f"Processed: {row['processed_text']}...")
    print(f"Lemmas:        {row['lemma_text']}...")
    print(f"POS Tags:      {row['pos_seq']}...\n")

print("""
Проблеми:
1. Бренди та англійські вставки: Назви техніки (gаlаху, Меіzu, Хіаоmі) та англійські слова стабільно не розпізнаються, отримують тег 'X' (Other) і не лематизуються.
2. Російська мова: Україномовна модель Stanza абсолютно не справляється з повністю російськими відгуками (Приклади #48, #64). Більшість слів отримують тег 'X' або хибні "українізовані" леми (напр., 'чернуй' замість 'черную').
3. Суржик та одруківки: Слова на кшталт 'дрставка', 'празднік' залишаються як є, або отримують хибну лему ('пріч' замість 'причому').
4. Спецсимволи та дати: Модель надмірно токенізує дати (10.12.2017 -> 10 . 12 . 2017) та розбиває наш маскувальний тег з ЛР2 (<ID> -> < id >).

Чи критично це?
Як показав Baseline 2, ні! Незважаючи на те, що лематизатор створює шум на сміттєвих або іншомовних токенах, він чудово справляється з основною масою правильних українських слів. Саме тому лематизація загалом дала приріст +2.64% до Accuracy: модель сфокусувалася на коренях основних емоційних слів, ігноруючи шум.""")

Приклади помилок лематизації
--- Приклад #1 ---
Processed: 12.04.16 замовив МD818 (доставили швидко, тут питань немає), звичайно ж я уточнив пару раз оригінальний чи кабель, на що отримав позитивну. Розпакував отриманий кабель я не виявив написів на кабелі, заходимо на сайт Аррlе і читаємо що повинна бути напис Аn Аррlе Lіghtnіng tо usв саblе hаs "Dеsіgnеd bу Аррlе іn Саlіfоrnіа" аnd еіthеr "Аssеmblеd іn Сhіnа," "Аssеmbеd іn Vіеtnаm," оr " Іndústrіа Вrаsіlеіrа "оn thе саblе аbоut sеvеn іnсhеs frоm thе usв соnnесtоr. Дзвоню в Стилус, пояснюю що немає напису. Відповідають, що кабель оригінал і все. Я прошу якісь документи що підтверджують де був куплений цей кабель, негативна. Потім мені просто говорять "привозите здавайте". Втрачати 499 грн на китайському кабелі мені не хотілося (нехай він навіть МFі сертифікований). Привіз кабель в магазин. "Яка причина повернення?" запитують у мене, я відповідаю "ні лого Аррlе як на оригіналі", у "а, зрозуміло". Через три хвилини мені дають папірець і

In [ ]:
# Генерація audit_summary_lab3.md

audit_md = """# Audit Summary: Lab 3 (Lemma / POS Baseline)

## 1. Загальна інформація
* **Напрям:** Задача A (Класифікація текстів - Sentiment Analysis)
* **Інструмент:** Stanza (українська модель, процесори: tokenize, pos, lemma)

## 2. Результати Baseline-моделей
Використано TF-IDF (`max_features=1200`) + Logistic Regression.

* **Baseline 1 (без лем, processed_v2):**
  * Accuracy: 0.8484
  * Macro-F1: 0.8253
* **Baseline 2 (з лемами, lemma_text):**
  * Accuracy: 0.8748
  * Macro-F1: 0.8572
* **Baseline 3 (processed_v2 + POS n-grams):**
  * Accuracy: 0.8204
  * Macro-F1: 0.7865

**Різниця:** Лематизація покращила Accuracy на **+2.80%**, а додавання POS n-grams погіршило на **-2.64%**.

## 3. Аналіз помилок (Error Analysis)
Під час перевірки лематизації (Stanza) виявлено такі систематичні проблеми:
1. **Бренди та англійські вставки:** Назви техніки (gаlаху, Меіzu, Хіаоmі) та англійські слова стабільно не розпізнаються, отримують тег `X` (Other) і не лематизуються.
2. **Російська мова:** Україномовна модель Stanza не справляється з російськими відгуками. Більшість слів отримують тег `X` або хибні "українізовані" леми (напр., 'чернуй' замість 'черную').
3. **Суржик та одруківки:** Слова на кшталт 'дрставка', 'празднік' залишаються як є, або отримують хибну лему ('пріч' замість 'причому').
4. **Спецсимволи та дати:** Модель надмірно токенізує дати (10.12.2017 -> 10 . 12 . 2017) та розбиває наш маскувальний тег з ЛР2 (`<ID>` -> `< id >`).
5. **Вплив на метрики:** Попри цей шум на неформатних токенах, лематизатор чудово справляється з основною масою правильних українських слів, що загалом дало позитивний приріст метрик.

## 4. Висновок:
У задачі класифікації (Sentiment Analysis) використання лематизованого тексту (`lemma_text`) суттєво покращило результати baseline-моделі, піднявши Accuracy на +2.80% (з 0.8484 до 0.8748). Це покращення відбулося завдяки зменшенню розрідженості простору ознак при жорстко обмеженому словнику (`max_features=1200`), оскільки лематизація успішно об'єднала різні форми емоційних слів у єдині сутності. Натомість використання POS-тегів у комбінації з текстом та біграмами (Baseline 3) призвело до помітного погіршення метрик (падіння Accuracy на -2.64%). Згенеровані граматичні n-грами витіснили з лімітованого словника семантично важливі слова, перетворившись на інформаційний "шум". Також аналіз помилок показав, що лематизатор Stanza часто генерував хибні леми або теги на повністю російськомовних відгуках, специфічних брендах та суржику, але загальному росту метрик це не зашкодило. Отже, фінальне рішення для проєкту — **"використовувати леми"** як основний вхідний текст для навчання класифікатора. Від інтеграції POS-тегів у фічі повністю відмовитись, оскільки вони не додають цінності у поточній конфігурації.
"""

audit_filename = "audit_summary_lab3.md"
with open(audit_filename, "w", encoding="utf-8") as f:
    f.write(audit_md)

print(f"Файл {audit_filename} згенеровано.")

# Автоматичне завантаження в Google Colab
import sys
if 'google.colab' in sys.modules:
    from google.colab import files
    files.download(audit_filename)

Файл audit_summary_lab3.md згенеровано.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>